In [ ]:
#uploading my rfm data from by local machine into colab

from google.colab import files
uploaded = files.upload()










Saving rfm.csv to rfm (2).csv


In [ ]:
import pandas as pd
import plotly.express as px
import plotly.io as pio
import plotly.graph_objects as go
pio.templates.default = "plotly_white"



# Now reading data from the cvs file uploaded (rfm_csv)

data = pd.read_csv('rfm.csv')

data.dtypes




CustomerID              int64
PurchaseDate           object
TransactionAmount     float64
ProductInformation     object
OrderID                 int64
Location               object
dtype: object

In [ ]:
from datetime import datetime


#calculate the Recency, Frequency, and Monetary values of the customers

# Convert 'PurchaseDate' to datetime
data['PurchaseDate'] = pd.to_datetime(data['PurchaseDate'])

# Calculate Recency
data['Recency'] = (datetime.now().date() - data['PurchaseDate'].dt.date).dt.days

data



,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency
0,8814,2023-11-04,943.31,Product C,890075,Tokyo,-22
1,2188,2023-11-04,463.70,Product A,176819,London,-22
2,4608,2023-11-04,80.28,Product A,340062,New York,-22
3,2559,2023-11-04,221.29,Product A,239145,London,-22
4,9482,2023-11-04,739.56,Product A,194545,Paris,-22
...,...,...,...,...,...,...,...
995,2970,2023-10-06,759.62,Product B,275284,London,7
996,6669,2023-10-06,941.50,Product C,987025,New York,7
997,8836,2023-10-06,545.36,Product C,512842,London,7
998,1440,2023-10-06,729.94,Product B,559753,Paris,7


In [ ]:
# Calculate Frequency
frequency_data = data.groupby('CustomerID')['OrderID'].count().reset_index()


frequency_data.rename(columns={'OrderID': 'Frequency'}, inplace=True)


data = data.merge(frequency_data, on='CustomerID', how='left')


data

,CustomerID,PurchaseDate,TransactionAmount,ProductInformation,OrderID,Location,Recency,Frequency_x,Frequency_y,Frequency
0,8814,2023-11-04,943.31,Product C,890075,Tokyo,-22,1000,1,1
1,2188,2023-11-04,463.70,Product A,176819,London,-22,1000,1,1
2,4608,2023-11-04,80.28,Product A,340062,New York,-22,1000,1,1
3,2559,2023-11-04,221.29,Product A,239145,London,-22,1000,1,1
4,9482,2023-11-04,739.56,Product A,194545,Paris,-22,1000,1,1
...,...,...,...,...,...,...,...,...,...,...
995,2970,2023-10-06,759.62,Product B,275284,London,7,1000,1,1
996,6669,2023-10-06,941.50,Product C,987025,New York,7,1000,1,1
997,8836,2023-10-06,545.36,Product C,512842,London,7,1000,1,1
998,1440,2023-10-06,729.94,Product B,559753,Paris,7,1000,1,1


In [ ]:
#Calculate Monetary Value

monetary_data = data.groupby('CustomerID')['TransactionAmount'].sum().reset_index()

monetary_data.rename(columns={'TransactionAmount': 'MonetaryValue'}, inplace=True)

data = data.merge(monetary_data, on='CustomerID', how='left')


print(data.head())


















   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-11-04             943.31          Product C   890075   
1        2188   2023-11-04             463.70          Product A   176819   
2        4608   2023-11-04              80.28          Product A   340062   
3        2559   2023-11-04             221.29          Product A   239145   
4        9482   2023-11-04             739.56          Product A   194545   

   Location  Recency  Frequency_x  Frequency_y  Frequency  MonetaryValue_x  \
0     Tokyo      -22         1000            1          1           943.31   
1    London      -22         1000            1          1           463.70   
2  New York      -22         1000            1          1            80.28   
3    London      -22         1000            1          1           221.29   
4     Paris      -22         1000            1          1           739.56   

   MonetaryValue_y  MonetaryValue  
0           943.31         943.3

In [ ]:
#Calculating RFM Scores
#Now let’s calculate the recency, frequency, and monetary scores:




#Define scoring criteria for each RFM value
recency_scores = [5, 4, 3, 2, 1]  # Higher score for lower recency (more recent)
frequency_scores = [1, 2, 3, 4, 5]  # Higher score for higher frequency
monetary_scores = [1, 2, 3, 4, 5]  # Higher score for higher monetary value

# Calculate RFM scores
data['RecencyScore'] = pd.cut(data['Recency'], bins=5, labels=recency_scores)
data['FrequencyScore'] = pd.cut(data['Frequency'], bins=5, labels=frequency_scores)
data['MonetaryScore'] = pd.cut(data['MonetaryValue'], bins=5, labels=monetary_scores)

In [ ]:
# Convert RFM scores to numeric type
data['RecencyScore'] = data['RecencyScore'].astype(int)
data['FrequencyScore'] = data['FrequencyScore'].astype(int)
data['MonetaryScore'] = data['MonetaryScore'].astype(int)

In [ ]:
# Calculate RFM score by combining the individual scores
data['RFM_Score'] = data['RecencyScore'] + data['FrequencyScore'] + data['MonetaryScore']

# Create RFM segments based on the RFM score
segment_labels = ['Low-Value', 'Mid-Value', 'High-Value']
data['Value Segment'] = pd.qcut(data['RFM_Score'], q=3, labels=segment_labels)



print(data.head())

   CustomerID PurchaseDate  TransactionAmount ProductInformation  OrderID  \
0        8814   2023-11-04             943.31          Product C   890075   
1        2188   2023-11-04             463.70          Product A   176819   
2        4608   2023-11-04              80.28          Product A   340062   
3        2559   2023-11-04             221.29          Product A   239145   
4        9482   2023-11-04             739.56          Product A   194545   

   Location  Recency  Frequency_x  Frequency_y  Frequency  MonetaryValue_x  \
0     Tokyo      -22         1000            1          1           943.31   
1    London      -22         1000            1          1           463.70   
2  New York      -22         1000            1          1            80.28   
3    London      -22         1000            1          1           221.29   
4     Paris      -22         1000            1          1           739.56   

   MonetaryValue_y  MonetaryValue  RecencyScore  FrequencyScore  \
0

In [ ]:
#RFM Segment Distribution
segment_counts = data['Value Segment'].value_counts().reset_index()
segment_counts.columns = ['Value Segment', 'Count']

pastel_colors = px.colors.qualitative.Pastel

# Create the bar chart
fig_segment_dist = px.bar(segment_counts, x='Value Segment', y='Count',
                          color='Value Segment', color_discrete_sequence=pastel_colors,
                          title='RFM Value Segment Distribution')

# Update the layout
fig_segment_dist.update_layout(xaxis_title='RFM Value Segment',
                              yaxis_title='Count',
                              showlegend=False)

# Show the figure
fig_segment_dist.show()

In [ ]:
# Create a new column for RFM Customer Segments
data['RFM Customer Segments'] = ''

# Assign RFM segments based on the RFM score
data.loc[data['RFM_Score'] >= 9, 'RFM Customer Segments'] = 'Champions'
data.loc[(data['RFM_Score'] >= 6) & (data['RFM_Score'] < 9), 'RFM Customer Segments'] = 'Potential Loyalists'
data.loc[(data['RFM_Score'] >= 5) & (data['RFM_Score'] < 6), 'RFM Customer Segments'] = 'At Risk Customers'
data.loc[(data['RFM_Score'] >= 4) & (data['RFM_Score'] < 5), 'RFM Customer Segments'] = "Can't Lose"
data.loc[(data['RFM_Score'] >= 3) & (data['RFM_Score'] < 4), 'RFM Customer Segments'] = "Lost"

# Print the updated data with RFM segments
print(data[['CustomerID', 'RFM Customer Segments']])

     CustomerID RFM Customer Segments
0          8814   Potential Loyalists
1          2188   Potential Loyalists
2          4608   Potential Loyalists
3          2559   Potential Loyalists
4          9482   Potential Loyalists
..          ...                   ...
995        2970   Potential Loyalists
996        6669   Potential Loyalists
997        8836   Potential Loyalists
998        1440   Potential Loyalists
999        4759   Potential Loyalists

[1000 rows x 2 columns]


In [ ]:
#Analyzing the distribution of customers across different RFM customer segments within each value segment

segment_product_counts = data.groupby(['Value Segment', 'RFM Customer Segments']).size().reset_index(name='Count')

segment_product_counts = segment_product_counts.sort_values('Count', ascending=False)

fig_treemap_segment_product = px.treemap(segment_product_counts,
                                         path=['Value Segment', 'RFM Customer Segments'],
                                         values='Count',
                                         color='Value Segment', color_discrete_sequence=px.colors.qualitative.Pastel,
                                         title='RFM Customer Segments by Value')
fig_treemap_segment_product.show()


In [ ]:
#analyze the distribution of RFM values within the Champions segment:


# Filter the data to include only the customers in the Champions segment
champions_segment = data[data['RFM Customer Segments'] == 'Champions']

fig = go.Figure()
fig.add_trace(go.Box(y=champions_segment['RecencyScore'], name='Recency'))
fig.add_trace(go.Box(y=champions_segment['FrequencyScore'], name='Frequency'))
fig.add_trace(go.Box(y=champions_segment['MonetaryScore'], name='Monetary'))

fig.update_layout(title='Distribution of RFM Values within Champions Segment',
                  yaxis_title='RFM Value',
                  showlegend=True)

fig.show()

In [ ]:
#analyzing the correlation of the recency, frequency, and monetary scores within the champions segment:
correlation_matrix = champions_segment[['RecencyScore', 'FrequencyScore', 'MonetaryScore']].corr()

# Visualize the correlation matrix using a heatmap
fig_heatmap = go.Figure(data=go.Heatmap(
                   z=correlation_matrix.values,
                   x=correlation_matrix.columns,
                   y=correlation_matrix.columns,
                   colorscale='RdBu',
                   colorbar=dict(title='Correlation')))

fig_heatmap.update_layout(title='Correlation Matrix of RFM Values within Champions Segment')

fig_heatmap.show()

In [ ]:
#number of customers in all the segments

import plotly.colors

pastel_colors = plotly.colors.qualitative.Pastel

segment_counts = data['RFM Customer Segments'].value_counts()

# Create a bar chart to compare segment counts
fig = go.Figure(data=[go.Bar(x=segment_counts.index, y=segment_counts.values,
                            marker=dict(color=pastel_colors))])

# Set the color of the Champions segment as a different color
champions_color = 'rgb(158, 202, 225)'
fig.update_traces(marker_color=[champions_color if segment == 'Champions' else pastel_colors[i]
                                for i, segment in enumerate(segment_counts.index)],
                  marker_line_color='rgb(8, 48, 107)',
                  marker_line_width=1.5, opacity=0.6)

# Update the layout
fig.update_layout(title='Comparison of RFM Segments',
                  xaxis_title='RFM Segments',
                  yaxis_title='Number of Customers',
                  showlegend=False)

fig.show()

In [ ]:
#analyzing the recency, frequency, and monetary scores of all the segments:

#Calculate the average Recency, Frequency, and Monetary scores for each segment
segment_scores = data.groupby('RFM Customer Segments')['RecencyScore', 'FrequencyScore', 'MonetaryScore'].mean().reset_index()

# Create a grouped bar chart to compare segment scores
fig = go.Figure()

# Add bars for Recency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['RecencyScore'],
    name='Recency Score',
    marker_color='rgb(158,202,225)'
))

# Add bars for Frequency score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['FrequencyScore'],
    name='Frequency Score',
    marker_color='rgb(94,158,217)'
))



# Add bars for Monetary score
fig.add_trace(go.Bar(
    x=segment_scores['RFM Customer Segments'],
    y=segment_scores['MonetaryScore'],
    name='Monetary Score',
    marker_color='rgb(32,102,148)'
))

# Update the layout
fig.update_layout(
    title='Comparison of RFM Segments based on Recency, Frequency, and Monetary Scores',
    xaxis_title='RFM Segments',
    yaxis_title='Score',
    barmode='group',
    showlegend=True
)

fig.show()

<ipython-input-130-18a821e7a4e3>:4: FutureWarning:

Indexing with multiple keys (implicitly converted to a tuple of keys) will be deprecated, use a list instead.

